In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/athletes_clean.txt", sep = "\t")

In [3]:
df.head(1)

,Name,Games,Games_Order,Games_Year,is_summer,Host,Sport,Event,Num_Pos,Medal,is_team_event,Team,Male,Birth,Age,NOC,Country,is_at_home
0,Jean-François Blanchy,1912 Summer Olympics,5.0,1912,1,Sweden,Tennis,"Singles, Men",17.0,NaN,0,NaN,1,1886.0,26.0,FRA,France,0


In [4]:
"""
# Most Medals of Single Type Awarded
m_num = 0
m_arr = []
for k, k_df in df[df.is_team_event == 0].groupby(["Games", "Sport", "Event"]):
    golds = 0
    silvers = 0
    bronzes = 0
    for m in k_df.Medal:
        if m == "Gold":
            golds += 1
        elif m == "Silver":
            silvers += 1
        elif m == "Bronze":
            bronzes += 1
    if max(golds, silvers, bronzes) == m_num:
        m_arr.append(k)
    if max(golds, silvers, bronzes) > m_num:
        m_num = max(golds, silvers, bronzes)
        m_arr = [k]
print(m_num)
print(m_arr)
"""

"""
# Most Medals Awarded in Single Event 
m_num = 0
m_arr = []
for k, k_df in df[df.is_team_event == 0].groupby(["Games", "Sport", "Event"]):
    golds = 0
    silvers = 0
    bronzes = 0
    for m in k_df.Medal:
        if m == "Gold":
            golds += 1
        elif m == "Silver":
            silvers += 1
        elif m == "Bronze":
            bronzes += 1
    if golds + silvers + bronzes == m_num:
        m_arr.append(k)
    if golds + silvers + bronzes > m_num:
        m_num = golds + silvers + bronzes
        m_arr = [k]
print(m_num)
print(m_arr)
"""

'\n# Most Medals Awarded in Single Event \nm_num = 0\nm_arr = []\nfor k, k_df in df[df.is_team_event == 0].groupby(["Games", "Sport", "Event"]):\n    golds = 0\n    silvers = 0\n    bronzes = 0\n    for m in k_df.Medal:\n        if m == "Gold":\n            golds += 1\n        elif m == "Silver":\n            silvers += 1\n        elif m == "Bronze":\n            bronzes += 1\n    if golds + silvers + bronzes == m_num:\n        m_arr.append(k)\n    if golds + silvers + bronzes > m_num:\n        m_num = golds + silvers + bronzes\n        m_arr = [k]\nprint(m_num)\nprint(m_arr)\n'

In [5]:
# Assign Place Distance from Marginal Medal Finishing Place
df["dist_marginal_medal"] = np.nan
for k, g_df in df.groupby(["Games", "Sport", "Event"]):
    # Find Marginal Medalist Place
    marg_medal_place = np.nan
    if "Bronze" in set(g_df.Medal):
        marg_medal_place = max(g_df[g_df.Medal == "Bronze"].Num_Pos)
    elif "Silver" in set(g_df.Medal):
        marg_medal_place = max(g_df[g_df.Medal == "Silver"].Num_Pos)
    elif "Gold" in set(g_df.Medal):
        marg_medal_place = max(g_df[g_df.Medal == "Gold"].Num_Pos)
    else:
        print("NO MEDAL!")
        print(k)
    # Assign difference of Marginal Place and Num_Pos
    if np.isnan(marg_medal_place):
        continue
    df.loc[(df.Games == k[0]) & (df.Sport == k[1]) & (df.Event == k[2]), "dist_marginal_medal"] = marg_medal_place - g_df.Num_Pos

NO MEDAL!
('1900 Summer Olympics', 'Archery', 'Unknown Event, Men')
NO MEDAL!
('1912 Summer Olympics', 'Art Competitions', 'Literature, Open')
NO MEDAL!
('1920 Summer Olympics', 'Sailing', '18 foot, Open')
NO MEDAL!
('1920 Summer Olympics', 'Shooting', 'Unknown Event, Men')
NO MEDAL!
('1924 Summer Olympics', 'Art Competitions', 'Literature, Open')
NO MEDAL!
('1924 Summer Olympics', 'Art Competitions', 'Sculpturing, Open')
NO MEDAL!
('1928 Summer Olympics', 'Art Competitions', 'Architecture, Unknown Event, Open')
NO MEDAL!
('1928 Summer Olympics', 'Art Competitions', 'Painting, Drawings And Water Colors, Open')
NO MEDAL!
('1928 Summer Olympics', 'Art Competitions', 'Painting, Paintings, Open')
NO MEDAL!
('1928 Winter Olympics', 'Speed Skating (Skating)', '10,000 metres, Men')
NO MEDAL!
('1932 Summer Olympics', 'Art Competitions', 'Architecture, Unknown Event, Open')
NO MEDAL!
('1932 Summer Olympics', 'Art Competitions', 'Literature, Open')
NO MEDAL!
('1932 Summer Olympics', 'Art Competi

In [7]:
# Part I: Create Athletes Debut Games Dataset
# An individual is identified uniquely by ("Name", "Birth", "Male")
p1_df = df.loc[df.groupby(["Name", "Birth", "Male"]).Games_Order.idxmin()]
# Create athlete debut year mapping
names = list(p1_df.Name)
births = list(p1_df.Birth)
male = list(p1_df.Male)
debuts = list(p1_df.Games)
deb_dict = {(names[i], births[i], male[i]): debuts[i] for i in range(len(debuts))}

In [8]:
# Create Games_Order to Host mapping
order_host_dict = {}
for k, _ in df.groupby(["Games_Order", "Host", "is_summer", "Games_Year"]):
    order_host_dict[k[0]] = (k[1], k[2], k[3])

In [12]:
# Generate FutureOlympics, FutureMedals, FutureGolds, FutureSilvers, FutureBronzes, Events, TeamEvent, IndivEvent, BestIndivMarg, FutureHomeSame, FutureHomeDiff
fo = []
fm = []
fg = []
fs = []
fb = []
ev = []
tev = []
iev = []
bim = []
bmed = []
fhs = []
fhd = []
for k, g_df in df.groupby(["Name", "Birth", "Male"]):
    fo.append(len(g_df.Games.unique()) - 1)
    golds = 0
    silvers = 0
    bronzes = 0
    for m in g_df[g_df.Games != deb_dict[k]].Medal:
        if m == "Gold":
            golds += 1
        elif m == "Silver":
            silvers += 1
        elif m == "Bronze":
            bronzes += 1
    fm.append(golds + silvers + bronzes)
    fg.append(golds)
    fs.append(silvers)
    fb.append(bronzes)
    deb_df = g_df[g_df.Games == deb_dict[k]]
    ev.append(len(deb_df))
    tev.append(max(deb_df.is_team_event))
    iev.append(1 - min(deb_df.is_team_event))
    d_game_order = list(deb_df.Games_Order)[0]
    d_game_nat = list(deb_df.Country)[0]
    d_game_year = list(deb_df.Games_Year)[0]
    d_game_season = list(deb_df.is_summer)[0]
    same_h = 0
    diff_h = 0
    d_game_order += 1
    while d_game_order in order_host_dict and order_host_dict[d_game_order][2] - d_game_year <= 12:
        if d_game_nat == order_host_dict[d_game_order][0]:
            if order_host_dict[d_game_order][1] == d_game_season:
                same_h += 1
            else:
                diff_h += 1
        d_game_order += 1
    fhs.append(same_h)
    fhd.append(diff_h)
    i_deb_df = deb_df[deb_df.is_team_event == 0]
    if len(i_deb_df) == 0:
        bim.append(np.nan)
    else:
        bim.append(max(i_deb_df.dist_marginal_medal))
    med = np.nan
    for m in i_deb_df.Medal:
        if m == "Gold":
            med = "Gold"
        elif m == "Silver" and (med == "Bronze" or pd.isnull(med)):
            med = "Silver"
        elif m == "Bronze" and pd.isnull(med):
            med = "Bronze"
    bmed.append(med)
p1_df["Future_Olympics"] = fo
p1_df["Future_Medals"] = fm
p1_df["Future_Golds"] = fg
p1_df["Future_Silvers"] = fs
p1_df["Future_Bronzes"] = fb
p1_df["Events"] = ev
p1_df["Team_Event"] = tev
p1_df["Indiv_Event"] = iev
p1_df["Best_Indiv_Marg"] = bim
p1_df["Best_Indiv_Medal"] = bmed
p1_df["Future_Home_Same"] = fhs
p1_df["Future_Home_Diff"] = fhd

In [13]:
# Create Age_Sq
p1_df["Age_Sq"] = p1_df["Age"]**2
p1_df = p1_df[["Name", "Games", "Games_Year", "is_summer", "Host", "Male", "Birth", "NOC", "Country", "Age", "Age_Sq", "Future_Olympics", "Future_Medals", "Future_Golds", "Future_Silvers", "Future_Bronzes", "Events", "Team_Event", "Indiv_Event", "Best_Indiv_Marg", "Best_Indiv_Medal", "Future_Home_Same", "Future_Home_Diff"]]
p1_df.to_csv("../data/p1_athlete_debuts.txt", index = False, sep = "\t")